In [ ]:
# QUESTION 5

In [34]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from ca_utils import ResNet, BasicBlock
import numpy as np

# Define the model
num_classes = 10
model = ResNet(BasicBlock, [1,1,1], num_classes=num_classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load the saved model weights
model_path = 'Desktop/weights_resnet (1).pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() 

# Define transformations for the validation set
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the validation dataset
val_dataset = datasets.ImageFolder(root='Desktop/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define the test_cnn function to evaluate the model
def test_cnn(model, val_loader):
    model.eval() 
    correct = 0
    total = 0
    predictions = []

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            predictions.extend(predicted.cpu().numpy())

    accuracy = correct / total
    return np.array(predictions, dtype=np.int64), accuracy

# Run the test_cnn function and print the accuracy
predictions, accuracy = test_cnn(model, val_loader)
print(f'Accuracy of the model on the validation images: {accuracy * 100:.2f}%')
print (predictions)


Accuracy of the model on the validation images: 65.35%
[0 0 0 ... 9 9 8]


In [ ]:
# QUESTION 6

In [35]:
import torch

true_labels = []
for _, labels in val_loader:
    true_labels.extend(labels.tolist())
true_labels_array = torch.tensor(true_labels)

In [36]:
def compute_confusion_matrix(true, predictions):


    # Identify unique classes
    classes = np.unique(np.concatenate((true, predictions)))
    num_classes = len(classes)
    
    # Create mapping of class labels to indices
    class_to_index = dict((c, i) for i, c in enumerate(classes))
    
    # Initialize the confusion matrix
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)
    
    # Populate the confusion matrix
    for t, p in zip(true, predictions):
        true_index = class_to_index[t]
        pred_index = class_to_index[p]
        confusion_matrix[true_index, pred_index] += 1

    return confusion_matrix

cm = compute_confusion_matrix(true_labels, predictions)
print(cm)

[[163  10  11   0   1   0   1   0   7   7]
 [ 14 116   1  16  11  13   2  26   0   1]
 [  1   3 170   0   0   0   4   0   0  22]
 [  1  23   2  80  29  21   8  34   1   1]
 [  2  13   0  14 110   9  24  26   1   1]
 [  0   8   0  20  23  74  34  40   0   1]
 [  1   3   2   2  12  21 151   7   0   1]
 [  0  12   0   9  13  18   4 144   0   0]
 [ 16   4   2   2   0   1   0   0 150  25]
 [  5   0  23   3   1   0   7   0  12 149]]
